In [1]:
import pickle as pkl
import numpy as np
import pandas as pd
import sklearn
from twitter_nlp_toolkit.twit_module import twit_module
from twitter_nlp_toolkit.file_fetcher import file_fetcher
from zipfile import ZipFile

Using TensorFlow backend.


In [2]:
chunk = 1 # Fraction of data to train on
model_name = 'model_test_001'


Here we download some training and validation data.

The training data is the semi-supervised Sentiment140 dataset, taken form here: https://www.kaggle.com/kazanova/sentiment140

The validation data is hand-labeled airline customer feedback taken from https://www.figure-eight.com/data-for-everyone/

In [3]:
downloader = file_fetcher.downloader(using_notebook=True)

downloader initialized


In [4]:
# Validation data
downloader.download_file('https://www.dropbox.com/s/440m6x07bjg6c0h/Tweets.zip?dl=1',"Tweets.zip")

Downloading: 100% [1089469 / 1089469] bytes            


In [5]:
# Compressed model
# Note that this model has only been trained on 1% of the training data
# Update when better-trained model is available
downloader.download_file("https://www.dropbox.com/s/owpldku3kk7aaqj/model_test_001.zip?dl=1","model_test_001.zip")

Downloading: 99% [931241984 / 933152569] bytes            


In [6]:
#unzip the pickle file
with ZipFile('model_test_001.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

In [7]:
test_data = pd.read_csv('Tweets.zip', header=0, names=['Index', 'Sentiment', 'Sentiment_confidence',
                                                                'Negative_reason', 'Negative_reason_confidence',
                                                                'Airline', 'Airline_sentiment_gold', 'Handle',
                                                                'Negative_reason_gold', 'Retweet_count', 'Text',
                                                                'Tweet_coord', 'Time', 'Location', 'Timezone'])

In [8]:
test_data['Labels'] = (test_data['Sentiment'] == 'positive') * 2
test_data['Labels'] = test_data['Labels'] + (test_data['Sentiment'] == 'neutral') * 1
test_data['Labels'] = test_data['Labels'] / 2

# Remove unlabeled test data
test_data.set_index('Labels')
test_data = test_data[test_data.Labels != 0.5]

In [9]:
# Executing this cell takes about 20min on a laptop

Classifier = twit_module.SentimentAnalyzer()
Classifier.load_models(model_name)

No BoW model found
No LSTM model found
No pre-trained embedding model found


In [11]:
test_data['Labels'] = (test_data['Sentiment'] == 'positive') * 2

print("Test Accuracy: %.3f" % sklearn.metrics.accuracy_score(test_data['Labels']/2, Classifier.predict(test_data['Text']).reshape(-1)))

Test Accuracy: 0.685


In [12]:
Classifier.predict(['I am happy', 'I am sad', 'I am cheerful', 'I am mad'])

array([1., 0., 1., 1.])